In [ ]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt2
import seaborn as sns
import matplotlib as mpl
# import warnings
# warnings.filterwarnings("ignore")
import os
import skimage
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial
import squidpy as sq
from PIL import Image
from PIL.TiffTags import TAGS
from skimage import io
import tifffile as tf
import scipy
print(skimage.__version__) # make sure version is 0.19.2
# from cellpose import plot
# from cellpose import models # my modfied code 
# from cellpose import io
#import cv2
import matplotlib.pyplot as plt

In [ ]:
path_to_reptract = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/'

## Read in cell2location deconvoluted object

In [ ]:
cell2loc_metadata = sc.read('/lustre/scratch126/cellgen/team292/vl6/VISIUM/cell2location_males/male/predmodel/sp.h5ad')
cell2loc_metadata

In [ ]:
cell2loc_metadata.obs[cell2loc_metadata.uns['mod']['factor_names']] = cell2loc_metadata.obsm['q05_cell_abundance_w_sf']

In [ ]:
cell2loc_metadata.obs['tot_cell_abundance'] = cell2loc_metadata.uns["mod"]["post_sample_means"]["w_sf"].sum(1).flatten()
cell2loc_metadata.obs['detection_sensit']  = cell2loc_metadata.uns["mod"]["post_sample_q05"]["detection_y_s"]

In [ ]:
cell2loc_metadata = cell2loc_metadata[[i >= 5 for i in cell2loc_metadata.obs['tot_cell_abundance']]]

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, fontsize=14, 
                     figsize=[6,6], color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
cell2loc_metadata.obs['broad_anatomy_annots'].value_counts()

In [ ]:
cell2loc_metadata.var.head()

In [ ]:
path = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/'

In [ ]:
sample_IDs = ['HCA_F_RepTsp14733322', 'HCA_F_RepTsp14707486']

In [ ]:
adatas = []
for sample in sample_IDs:
    print(sample)
    adata = sc.read(path+sample+'/'+sample+'_annotated.h5ad')
    adata.var_names_make_unique()
    adata.obs['sample'] = sample
    adata.var['SYMBOL'] = adata.var_names
    #adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    #adata.var_names = adata.var['ENSEMBL']
    #adata.var.drop(columns='ENSEMBL', inplace=True)

    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var['SYMBOL']]
    adata.obs['mt_frac'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']

    # mitochondria-encoded (MT) genes should be removed for spatial mapping
    adata.obsm['mt'] = adata[:, adata.var['mt'].values].X.toarray()
    adata = adata[:, ~adata.var['mt'].values]

    # add sample name to obs names
    adata.obs["sample"] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs["sample"] \
                          + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    adata = adata[[i == 1.0 for i in adata.obs['in_tissue']]]
    print(adata.shape)
    adatas.append(adata)

In [ ]:
for adata in adatas:
    if 'x' in adata.obs.columns.to_list():
        del adata.obs['x']

In [ ]:
for adata in adatas:
    if 'y' in adata.obs.columns.to_list():
        del adata.obs['y']

In [ ]:
adata_cytassist = sc.concat(
    adatas,
    join = "outer",
    uns_merge="unique",
    keys=[
        k
        for d in [
            adatas[0].uns["spatial"],
            adatas[1].uns["spatial"],
        ]
        for k, v in d.items()
    ],
    index_unique=None,
)
adata_cytassist.var_names_make_unique()

In [ ]:
adata_cytassist.obs['sample'].value_counts()

In [ ]:
for c in cell2loc_metadata.obs.columns.to_list():
    if c not in adata_cytassist.obs.columns.to_list():
        print(c)
        adata_cytassist.obs[c] = adata_cytassist.obs_names.map(cell2loc_metadata.obs[c].to_dict())

In [ ]:
adata_cytassist.obs['longitudinal_anatomy'].value_counts(0)

In [ ]:
adata_cytassist.obs['tot_cell_abundance']

In [ ]:
def select_slide(adata, s, batch_key="sample"):
    r"""This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param batch_key: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[batch_key].isin([s]), :].copy()
    s_keys = list(slide.uns["spatial"].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns["spatial"] = {s_spatial: slide.uns["spatial"][s_spatial]}

    return slide

## 1. Build epididymal axis

In [ ]:
def transpose_axis(orig_axis, new_min, new_max):
    min_val = min(orig_axis)
    max_val = max(orig_axis)
    # Apply linear transformation
    dest_axis = [(x - min_val) / (max_val - min_val) * (new_max - new_min) + new_min for x in orig_axis]
    
    return dest_axis

In [ ]:
def build_epididymal_axis(adata, sample, min_cells, start_feature, end_feature, start_weight = 1, end_weight = 1,
                         excluded_regions = ['ToExclude', 'Background'], is_stitched = False, sample1 = None, sample2 = None):
    
    print('Is the sample stitched? {}'.format(is_stitched))
    
    if not is_stitched:
        
        print('Building uterovaginal axis for sample {} from {} (weight: {}) to {} (weight: {})'.format(sample, start_feature, 
                            start_weight, end_feature, end_weight))
        
        # Select slide from spatial anndata object
        adata_sample = select_slide(adata, sample, batch_key = 'sample')
        adata_sample = adata_sample[[i == 'Epididymis' for i in adata_sample.obs['broad_anatomy_annots']]]
        
        # Remove excluded regions 
        adata_sample = adata_sample[[i not in excluded_regions for i in adata_sample.obs['longitudinal_anatomy']]]
        adata_sample = adata_sample[[i > min_cells for i in adata_sample.obs['tot_cell_abundance']]]
    
        adata_sample = adata_sample[[notna_val for notna_val in adata_sample.obs['L2_dist_longitudinal_anatomy_EfferentDuctules'].notna()]]
        
        # Build axis 
        df = adata_sample.obs[['L2_dist_longitudinal_anatomy_' + start_feature, 'L2_dist_longitudinal_anatomy_' + end_feature, 
               'broad_anatomy_annots']]
        print(df.head())
        print(df['L2_dist_longitudinal_anatomy_' + start_feature].isna().value_counts())
        epididymal_axis = [( start_weight*int(s) - end_weight*int(e) ) / ( start_weight*int(s) + end_weight*int(e) ) for s,e in zip(df['L2_dist_longitudinal_anatomy_' + start_feature], df['L2_dist_longitudinal_anatomy_' + end_feature])]
        df['Epididymal_Axis'] = epididymal_axis
        
        df['Epididymal_Axis'] = transpose_axis(df['Epididymal_Axis'], -2, 0)
        print(df['Epididymal_Axis'].hist(bins=20))
        
        # Add axis to slide 
        adata_sample.obs['Epididymal_Axis'] = adata_sample.obs_names.map(df['Epididymal_Axis'].to_dict())
        sc.pl.spatial(adata_sample, color="Epididymal_Axis", cmap = 'jet')
        print(df['Epididymal_Axis'].hist(bins=20))
    
        
        return adata_sample 

In [ ]:
HCA_F_RepTsp14733322_axis = build_epididymal_axis(adata_cytassist, 'HCA_F_RepTsp14733322', 10, 'EfferentDuctules', 'EpididymisEnd', 
                                                                      start_weight = 0.5, end_weight = 0.5, is_stitched = False)

In [ ]:
HCA_F_RepTsp14733322 = select_slide(adata_cytassist, 'HCA_F_RepTsp14733322', batch_key = 'sample')

In [ ]:
HCA_F_RepTsp14733322.obs['Epididymal_Axis'] = HCA_F_RepTsp14733322.obs_names.map(HCA_F_RepTsp14733322_axis.obs['Epididymal_Axis'].to_dict())

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, fontsize=14, 
                     figsize=[6,6], color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
from matplotlib.colors import LinearSegmentedColormap


# Define the colors and their corresponding positions
colors = ["#34d5e4", "#55b0ff", "#11006e"]

# Create the colormap
cmap_epididymis = LinearSegmentedColormap.from_list('custom_cmap', colors, N=256)

In [ ]:
sc.pl.spatial(HCA_F_RepTsp14733322, color = 'Epididymal_Axis', cmap = cmap_epididymis, 
             save = 'HCA_F_RepTsp14733322_epididymal_axis')

In [ ]:
sc.pl.spatial(HCA_F_RepTsp14733322, 
             save = 'HCA_F_RepTsp14733322_h&e')

In [ ]:
HCA_F_RepTsp14707486_axis = build_epididymal_axis(adata_cytassist, 'HCA_F_RepTsp14707486', 10, 'EfferentDuctules', 'EpididymisEnd', 
                                                                      start_weight = 0.5, end_weight = 0.5, is_stitched = False)

## 2. Build vas deferens axis

In [ ]:
adata_cytassist.obs['longitudinal_anatomy'].value_counts()

In [ ]:
def build_vasdeferens_axis(adata, sample, min_cells, start_feature, end_feature, start_weight = 1, end_weight = 1,
                         excluded_regions = ['ToExclude', 'Background'], is_stitched = False, sample1 = None, sample2 = None):
    
    print('Is the sample stitched? {}'.format(is_stitched))
    
    if not is_stitched:
        
        print('Building uterovaginal axis for sample {} from {} (weight: {}) to {} (weight: {})'.format(sample, start_feature, 
                            start_weight, end_feature, end_weight))
        
        # Select slide from spatial anndata object
        adata_sample = select_slide(adata, sample, batch_key = 'sample')
        adata_sample = adata_sample[[i == 'VasDeferens' for i in adata_sample.obs['broad_anatomy_annots']]]
        
        # Remove excluded regions 
        adata_sample = adata_sample[[i not in excluded_regions for i in adata_sample.obs['longitudinal_anatomy']]]
        adata_sample = adata_sample[[i > min_cells for i in adata_sample.obs['tot_cell_abundance']]]
    
        adata_sample = adata_sample[[notna_val for notna_val in adata_sample.obs['L2_dist_longitudinal_anatomy_VasDeferensStart'].notna()]]
        adata_sample = adata_sample[[notna_val for notna_val in adata_sample.obs['L2_dist_longitudinal_anatomy_VasDeferensEnd'].notna()]]
        
        # Build axis 
        df = adata_sample.obs[['L2_dist_longitudinal_anatomy_' + start_feature, 'L2_dist_longitudinal_anatomy_' + end_feature, 
               'broad_anatomy_annots']]
        print(df.head())
        print(df['L2_dist_longitudinal_anatomy_' + start_feature].isna().value_counts())
        epididymal_axis = [( start_weight*int(s) - end_weight*int(e) ) / ( start_weight*int(s) + end_weight*int(e) ) for s,e in zip(df['L2_dist_longitudinal_anatomy_' + start_feature], df['L2_dist_longitudinal_anatomy_' + end_feature])]
        df['VasDeferens_Axis'] = epididymal_axis
        
        df['VasDeferens_Axis'] = transpose_axis(df['VasDeferens_Axis'], 0, 2)
        print(df['VasDeferens_Axis'].hist(bins=20))
        
        # Add axis to slide 
        adata_sample.obs['VasDeferens_Axis'] = adata_sample.obs_names.map(df['VasDeferens_Axis'].to_dict())
        sc.pl.spatial(adata_sample, color="VasDeferens_Axis", cmap = 'jet')
        print(df['VasDeferens_Axis'].hist(bins=20))
    
        
        return adata_sample 

In [ ]:
HCA_F_RepTsp14733322_vd_axis = build_vasdeferens_axis(adata_cytassist, 'HCA_F_RepTsp14733322', 10, 'VasDeferensStart', 'VasDeferensEnd', 
                                                                      start_weight = 0.5, end_weight = 0.5, is_stitched = False)

In [ ]:
HCA_F_RepTsp14707486_vd_axis = build_vasdeferens_axis(adata_cytassist, 'HCA_F_RepTsp14707486', 10, 'VasDeferensStart', 'VasDeferensEnd', 
                                                                      start_weight = 0.5, end_weight = 0.5, is_stitched = False)

## 3. Combine samples - epididymis

In [ ]:
def combine_axis(samples, axis_name, return_adata = False):
    
    # Merge anndata objects
    adata_all = samples[0].concatenate(samples[1:], index_unique = None)
    
    # Remove spots that don't have a value for the axis 
    adata_all_obs = adata_all.obs
    adata_all_obs = adata_all_obs[adata_all_obs[axis_name].notna()]
    adata_all = adata_all[adata_all_obs.index.to_list(), :]
    
    if return_adata:
        return adata_all
    else:
        return adata_all_obs

In [ ]:
epididymal_axis = combine_axis(samples = [
                                           HCA_F_RepTsp14733322_axis,HCA_F_RepTsp14707486_axis
                                           ], 
                                axis_name = 'Epididymal_Axis', return_adata = True)

In [ ]:
epididymal_axis

In [ ]:
# Selecting relevant columns for computation
relevant_columns = ['Coelomic_Epithelium', 'CaputEpididymis_Epithelium', 'Corpus/CaudaEpididymis_Epithelium', 'CaputEpididymis_Ciliated_Epithelium', 'Corpus/CaputEpididymis_Mesenchyme', 'Corpus/CaputEpididymis_Ligament', 'CaudaEpididymis_Mesenchyme', 'CaudaEpididymis_Ligament', 'VasDeferens_Mesenchyme', 'VasDeferens_Ligament', 'Urethral_Plate', 'Genital_Epidermis', 'Preputial_Lamina', 'ExternalGenitalia_CorpusSpongiosum', 'ExternalGenitalia_CorpusCavernosum', 'ExternalGenitalia_Glans', 'ExternalGenitalia_Prepuce', 'ExternalGenitalia_SubdermalPrepuce', 'Pre-Perivascular', 'Perivascular', 'SkeletalMuscle', 'Endothelial', 'Endothelial_Lymphatic', 'Immune', 'Erythroid', 'SchwannCell', 'Neural']

# Calculating the percentage of cells that are of the most abundant cell type for each spot
epididymal_axis.obs['Most_Abundant_Cell_Type'] = epididymal_axis.obs[relevant_columns].idxmax(axis=1)
epididymal_axis.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
for s in epididymal_axis.obs_names:
    ct = epididymal_axis.obs.loc[s, 'Most_Abundant_Cell_Type']
    n_ct = epididymal_axis.obs.loc[s, ct]
    n_tot = epididymal_axis.obs.loc[s, relevant_columns].sum()
    epididymal_axis.obs.loc[s, 'Percentage_Most_Abundant_Cell_Type'] = (n_ct / n_tot) 

In [ ]:
epididymal_mese = epididymal_axis[[i in ['Corpus/CaputEpididymis_Mesenchyme', 
                                     ] for i in epididymal_axis.obs['Most_Abundant_Cell_Type']]]
epididymal_mese

In [ ]:
epididymal_mese.obs['Percentage_Most_Abundant_Cell_Type'].hist(bins = 20)

In [ ]:
epididymal_mese.var.head()

In [ ]:
epididymal_mese = epididymal_mese[[i > 0.7 for i in epididymal_mese.obs['Percentage_Most_Abundant_Cell_Type']]]

In [ ]:


HCA_F_RepTsp14707486_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_axis.obs_names.map(epididymal_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14707486_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_axis.obs_names.map(epididymal_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_axis, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp14733322_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_axis.obs_names.map(epididymal_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14733322_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_axis.obs_names.map(epididymal_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_axis, color="Most_Abundant_Cell_Type")

In [ ]:
epididymal_mese.X[20:30, 20:30].toarray()

In [ ]:
epididymal_mese_forR = epididymal_mese.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in epididymal_mese_forR.var_names if not name.startswith('MT-')]
epididymal_mese_forR = epididymal_mese_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in epididymal_mese_forR.var_names if not name.startswith('RP')]
epididymal_mese_forR = epididymal_mese_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in epididymal_mese_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
epididymal_mese_forR = epididymal_mese_forR[:, non_haemo_genes]

sc.pp.filter_genes(epididymal_mese_forR, min_cells = 10)
sc.pp.filter_genes(epididymal_mese_forR, min_counts = 1)

In [ ]:
epididymal_mese_forR.shape

In [ ]:
for col in epididymal_mese_forR.obs.columns:
    if col not in [ 'Epididymal_Axis', 'sample']:
        del epididymal_mese_forR.obs[col]

In [ ]:
epididymal_mese_forR.obs['Epididymal_Axis'].hist(bins = 10)

In [ ]:
del epididymal_mese_forR.obsm

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -i epididymal_mese_forR
epididymal_mese_forR

In [ ]:
%%R 
saveRDS(epididymal_mese_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/epididymalaxis_mese_cytassist.rds')

In [ ]:
epididymal_epi = epididymal_axis[[i in ['Corpus/CaputEpididymis_Mesenchyme', 'CaputEpididymis_Epithelium', 
                                         'Corpus/CaudaEpididymis_Epithelium'
                                     ] for i in epididymal_axis.obs['Most_Abundant_Cell_Type']]]
epididymal_epi

In [ ]:
epididymal_epi.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
epididymal_epi.obs['Corpus/CaudaEpididymis_Epithelium'].hist()

In [ ]:
epididymal_epi.obs['Epididymis_Epi_combined'] = epididymal_epi.obs['Corpus/CaudaEpididymis_Epithelium'] + epididymal_epi.obs['CaputEpididymis_Epithelium']


In [ ]:
epididymal_epi.obs['Epididymis_Epi_combined'].hist()

In [ ]:
epididymal_epi = epididymal_epi[[i >= 3 for i in epididymal_epi.obs['Epididymis_Epi_combined']]]


In [ ]:
epididymal_epi.shape

In [ ]:


HCA_F_RepTsp14707486_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_axis.obs_names.map(epididymal_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14707486_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_axis.obs_names.map(epididymal_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_axis, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp14733322_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_axis.obs_names.map(epididymal_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14733322_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_axis.obs_names.map(epididymal_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_axis, color="Most_Abundant_Cell_Type")

In [ ]:
epididymal_epi.obs['Epididymal_Axis'].hist(bins = 3)

In [ ]:
## Downsample per bin
def bin_axis(df, n_bins = 10, axis_name = 'Epididymal_Axis'):
    max_val = np.nanquantile(df[axis_name],0.9999)
    min_val = np.nanquantile(df[axis_name],0.0001)

    incr_uva = (max_val-min_val)/n_bins
    uva_bins = []
    for i in range(0,n_bins+1):
        uva_bins.append(min_val+(i*incr_uva))

    df['binned_' + axis_name + '_' + str(n_bins) + '_bins'] = pd.cut(df[axis_name], bins = uva_bins, labels=uva_bins[:-1])
    return df

In [ ]:
# Random downsampling per cell type 
import random
from itertools import chain
def downsample(adata, labels, n): 
    
    myindex = adata.obs[labels].value_counts().index 
    myvalues = adata.obs[labels].value_counts().values
    clusters = pd.Series(myvalues, index = myindex)
    
    # Find clusters with > n cells 
    cl2downsample = clusters.index[ clusters.values > n ]

    # save all barcode ids from small clusters
    holder = []
    holder.append( adata.obs_names[[ i not in cl2downsample for i in adata.obs[labels] ]] ) 

    # randomly sample n cells in the cl2downsample
    for cl in cl2downsample:
        print(cl)
        cl_sample = adata[[ i == cl for i in adata.obs[labels]]].obs_names
        
        cl_downsample = random.sample(set(cl_sample), n )
        holder.append(cl_downsample)
    
    # samples to include
    samples = list(chain(*holder))

    # Filter adata_count
    adata = adata[[ i in samples for i in adata.obs_names ]]
    return adata

In [ ]:
axis = bin_axis(epididymal_epi.obs, n_bins = 3, axis_name = 'Epididymal_Axis')
axis['binned_Epididymal_Axis_3_bins'] = axis['binned_Epididymal_Axis_3_bins'].astype(float)
axis['binned_Epididymal_Axis_3_bins'] = np.round(axis['binned_Epididymal_Axis_3_bins'], 2)
epididymal_epi.obs['binned_Epididymal_Axis_3_bins'] = epididymal_epi.obs_names.map(axis['binned_Epididymal_Axis_3_bins'].to_dict())

In [ ]:
epididymal_epi.obs['binned_Epididymal_Axis_3_bins'].value_counts()

In [ ]:
epididymal_epi_downsampled = downsample(epididymal_epi, 'binned_Epididymal_Axis_3_bins', 52)

In [ ]:
epididymal_epi_downsampled.shape

In [ ]:
epididymal_epi_forR = epididymal_epi_downsampled.copy()

In [ ]:
epididymal_epi_forR.shape

In [ ]:
# remove mito genes
non_mito_genes = [name for name in epididymal_epi_forR.var_names if not name.startswith('MT-')]
epididymal_epi_forR = epididymal_epi_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in epididymal_epi_forR.var_names if not name.startswith('RP')]
epididymal_epi_forR = epididymal_epi_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in epididymal_epi_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
epididymal_epi_forR = epididymal_epi_forR[:, non_haemo_genes]

# remove heat shock protein genes
non_hps_genes = [name for name in epididymal_epi_forR.var_names if not name.startswith('HSP')]
epididymal_epi_forR = epididymal_epi_forR[:, non_hps_genes]


sc.pp.filter_genes(epididymal_epi_forR, min_cells = 10)
sc.pp.filter_genes(epididymal_epi_forR, min_counts = 1)

In [ ]:
del epididymal_epi_forR.obsm

In [ ]:
%%R -i epididymal_epi_forR
epididymal_epi_forR

In [ ]:
%%R 
saveRDS(epididymal_epi_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/epididymalaxis_epi_downsampled_cytassist.rds')

## Plot selected genes from TradeSeq and prioritisation with scRNA-seq 

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(7, 7), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
spatially_variable_secretory_epi = ['CLDN2', 'CLDN10', 'CDH6', 'CDH2', 'AMN', 'FMO1', 'PDK4', 'SALL1',
                                   'SPAG11A', 'GLYAT', 'GLYATL1', 'SPINK2', 'PDZK1', 'PDZK1IP1',
                                    'ESR1', 'CFTR', 'VIL1', 'ACSL5', 'POMC', 'CPM', 'PCP4',
                                   'KRT17', 'WNT9B', 'PIK3C2G',  'BLNK', 'TFAP2A', 'GATA3', 'MARCH11', 'CPXM2', 'CXCL14', 'WNT6','MUC6',  
                                  ]

In [ ]:
# Create matrix plot using scanpy
sc.pl.matrixplot(epididymal_epi_downsampled,var_names=spatially_variable_secretory_epi,groupby='binned_Epididymal_Axis_3_bins',
                 num_categories = 3,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_epididymis_epi_genes_spatiallyvariable.pdf'
                )

## 4. Combine samples - vas deferens 

In [ ]:
vasdeferens_axis = combine_axis(samples = [
                                           HCA_F_RepTsp14733322_vd_axis,HCA_F_RepTsp14707486_vd_axis
                                           ], 
                                axis_name = 'VasDeferens_Axis', return_adata = True)

In [ ]:
vasdeferens_axis

In [ ]:
# Selecting relevant columns for computation
relevant_columns = ['Coelomic_Epithelium', 'CaputEpididymis_Epithelium', 'Corpus/CaudaEpididymis_Epithelium', 'CaputEpididymis_Ciliated_Epithelium', 'Corpus/CaputEpididymis_Mesenchyme', 'Corpus/CaputEpididymis_Ligament', 'CaudaEpididymis_Mesenchyme', 'CaudaEpididymis_Ligament', 'VasDeferens_Mesenchyme', 'VasDeferens_Ligament', 'Urethral_Plate', 'Genital_Epidermis', 'Preputial_Lamina', 'ExternalGenitalia_CorpusSpongiosum', 'ExternalGenitalia_CorpusCavernosum', 'ExternalGenitalia_Glans', 'ExternalGenitalia_Prepuce', 'ExternalGenitalia_SubdermalPrepuce', 'Pre-Perivascular', 'Perivascular', 'SkeletalMuscle', 'Endothelial', 'Endothelial_Lymphatic', 'Immune', 'Erythroid', 'SchwannCell', 'Neural']

# Calculating the percentage of cells that are of the most abundant cell type for each spot
vasdeferens_axis.obs['Most_Abundant_Cell_Type'] = vasdeferens_axis.obs[relevant_columns].idxmax(axis=1)
vasdeferens_axis.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
for s in vasdeferens_axis.obs_names:
    ct = vasdeferens_axis.obs.loc[s, 'Most_Abundant_Cell_Type']
    n_ct = vasdeferens_axis.obs.loc[s, ct]
    n_tot = vasdeferens_axis.obs.loc[s, relevant_columns].sum()
    vasdeferens_axis.obs.loc[s, 'Percentage_Most_Abundant_Cell_Type'] = (n_ct / n_tot) 

In [ ]:
vasdeferens_mese = vasdeferens_axis[[i in ['VasDeferens_Mesenchyme', 
                                     ] for i in vasdeferens_axis.obs['Most_Abundant_Cell_Type']]]
vasdeferens_mese

In [ ]:
vasdeferens_mese.obs['Percentage_Most_Abundant_Cell_Type'].hist(bins = 20)

In [ ]:
vasdeferens_mese = vasdeferens_mese[[i > 0.7 for i in vasdeferens_mese.obs['Percentage_Most_Abundant_Cell_Type']]]

In [ ]:


HCA_F_RepTsp14707486_vd_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_vd_axis.obs_names.map(vasdeferens_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_vd_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14707486_vd_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_vd_axis.obs_names.map(vasdeferens_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_vd_axis, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp14733322_vd_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_vd_axis.obs_names.map(vasdeferens_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_vd_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14733322_vd_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_vd_axis.obs_names.map(vasdeferens_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_vd_axis, color="Most_Abundant_Cell_Type")

In [ ]:
vasdeferens_mese.X[20:30, 20:30].toarray()

In [ ]:
vasdeferens_mese_forR = vasdeferens_mese.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in vasdeferens_mese_forR.var_names if not name.startswith('MT-')]
vasdeferens_mese_forR = vasdeferens_mese_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in vasdeferens_mese_forR.var_names if not name.startswith('RP')]
vasdeferens_mese_forR = vasdeferens_mese_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in vasdeferens_mese_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
vasdeferens_mese_forR = vasdeferens_mese_forR[:, non_haemo_genes]

sc.pp.filter_genes(vasdeferens_mese_forR, min_cells = 10)
sc.pp.filter_genes(vasdeferens_mese_forR, min_counts = 1)

In [ ]:
vasdeferens_mese_forR.shape

In [ ]:
for col in vasdeferens_mese_forR.obs.columns:
    if col not in [ 'VasDeferens_Axis', 'sample']:
        del vasdeferens_mese_forR.obs[col]

In [ ]:
vasdeferens_mese_forR.obs['VasDeferens_Axis'].hist(bins = 10)

In [ ]:
del vasdeferens_mese_forR.obsm

In [ ]:
%%R -i vasdeferens_mese_forR
vasdeferens_mese_forR

In [ ]:
%%R 
saveRDS(vasdeferens_mese_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/vasdeferensaxis_mese_cytassist.rds')

In [ ]:
vasdeferens_epi = vasdeferens_axis[[i in ['Corpus/CaudaEpididymis_Epithelium', 'CaputEpididymis_Epithelium', 
                                         'VasDeferens_Mesenchyme'
                                     ] for i in vasdeferens_axis.obs['Most_Abundant_Cell_Type']]]
vasdeferens_epi

In [ ]:
vasdeferens_epi.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
epididymal_epi.obs['Corpus/CaudaEpididymis_Epithelium'].hist()


In [ ]:
vasdeferens_epi = vasdeferens_epi[[i >= 3 for i in vasdeferens_epi.obs['Corpus/CaudaEpididymis_Epithelium']]]


In [ ]:
vasdeferens_epi.shape

In [ ]:
HCA_F_RepTsp14707486_vd_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_vd_axis.obs_names.map(vasdeferens_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_vd_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14707486_vd_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14707486_vd_axis.obs_names.map(vasdeferens_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14707486_vd_axis, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp14733322_vd_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_vd_axis.obs_names.map(vasdeferens_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_vd_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14733322_vd_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14733322_vd_axis.obs_names.map(vasdeferens_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14733322_vd_axis, color="Most_Abundant_Cell_Type")

In [ ]:
vasdeferens_epi_forR = vasdeferens_epi.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in vasdeferens_epi_forR.var_names if not name.startswith('MT-')]
vasdeferens_epi_forR = vasdeferens_epi_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in vasdeferens_epi_forR.var_names if not name.startswith('RP')]
vasdeferens_epi_forR = vasdeferens_epi_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in vasdeferens_epi_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
vasdeferens_epi_forR = vasdeferens_epi_forR[:, non_haemo_genes]

# remove heat shock protein genes
non_hps_genes = [name for name in vasdeferens_epi_forR.var_names if not name.startswith('HSP')]
vasdeferens_epi_forR = vasdeferens_epi_forR[:, non_hps_genes]


sc.pp.filter_genes(vasdeferens_epi_forR, min_cells = 10)
sc.pp.filter_genes(vasdeferens_epi_forR, min_counts = 1)

In [ ]:
del vasdeferens_epi_forR.obsm

In [ ]:
%%R -i vasdeferens_epi_forR
vasdeferens_epi_forR

In [ ]:
%%R 
saveRDS(vasdeferens_epi_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/vasdeferensaxis_epi_cytassist.rds')

## 5. Concatenate epididymis and vas deferens axes 

In [ ]:
vasdeferens_axis.obs['MaleReproductiveAxis'] = vasdeferens_axis.obs['VasDeferens_Axis'].copy()

In [ ]:
epididymal_axis.obs['MaleReproductiveAxis'] = epididymal_axis.obs['Epididymal_Axis'].copy()

In [ ]:
epididymal_axis = epididymal_axis.obs.copy()
vasdeferens_axis = vasdeferens_axis.obs.copy()

In [ ]:
axis = epididymal_axis.append(vasdeferens_axis)

In [ ]:
axis['MaleReproductiveAxis'].hist(bins = 6)

In [ ]:
axis.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/MaleReproductiveAxis_spots_cytassist.csv')

In [ ]:
for s in [HCA_F_RepTsp14707486_axis, HCA_F_RepTsp14733322_axis]:
    s.obs['MaleReproductiveAxis'] = s.obs_names.map(s.obs['Epididymal_Axis'])
    
for s in [HCA_F_RepTsp14707486_vd_axis, HCA_F_RepTsp14733322_vd_axis]:
    s.obs['MaleReproductiveAxis'] = s.obs_names.map(s.obs['VasDeferens_Axis'])

In [ ]:
axis_adata = combine_axis(samples = [
            HCA_F_RepTsp14707486_axis, HCA_F_RepTsp14733322_axis, 
   HCA_F_RepTsp14707486_vd_axis, HCA_F_RepTsp14733322_vd_axis
                                           ], 
                                axis_name = 'MaleReproductiveAxis', return_adata = True)

In [ ]:
axis_adata.obs['MaleReproductiveAxis'].hist(bins = 6)

In [ ]:
axis_adata.obs['MaleReproductiveAxis'] = axis_adata.obs_names.map(axis['MaleReproductiveAxis'].to_dict())

In [ ]:
epididymal_mese_spots = epididymal_mese.obs_names.to_list()
epididymal_epi_spots = epididymal_epi.obs_names.to_list()

In [ ]:
vasdeferens_epi_spots = vasdeferens_epi.obs_names.to_list()
vasdeferens_mese_spots = vasdeferens_mese.obs_names.to_list()

In [ ]:
len(epididymal_mese_spots)+ len(vasdeferens_mese_spots)

In [ ]:
epididymal_mese_spots.extend(vasdeferens_mese_spots)

In [ ]:
len(epididymal_mese_spots)

In [ ]:
len(epididymal_epi_spots) + len(vasdeferens_epi_spots)

In [ ]:
epididymal_epi_spots.extend(vasdeferens_epi_spots)

In [ ]:
axis_adata_mese = axis_adata[[i for i in axis_adata.obs_names.to_list() if i in epididymal_mese_spots], :]

In [ ]:
axis_adata_mese

In [ ]:
axis = bin_axis(axis_adata_mese.obs, n_bins = 6, axis_name = 'MaleReproductiveAxis')

In [ ]:
axis['binned_MaleReproductiveAxis_6_bins'] = axis['binned_MaleReproductiveAxis_6_bins'].astype(float)
axis['binned_MaleReproductiveAxis_6_bins'] = np.round(axis['binned_MaleReproductiveAxis_6_bins'], 2)
axis_adata_mese.obs['binned_MaleReproductiveAxis_6_bins'] = axis_adata_mese.obs_names.map(axis['binned_MaleReproductiveAxis_6_bins'].to_dict())

In [ ]:
axis_adata_mese.obs['binned_MaleReproductiveAxis_6_bins'].value_counts()

In [ ]:
axis_adata_mese.write('/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_mese_cytassist.h5ad')

In [ ]:
axis_adata_mese = downsample(axis_adata_mese, 'binned_MaleReproductiveAxis_6_bins', 47)

In [ ]:
axis_adata_mese.write('/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_mese_downsampled_cytassist.h5ad')

In [ ]:
axis_adata_mese.obs['MaleReproductiveAxis'].hist(bins = 6)

In [ ]:
axis_adata_mese.X[20:30, 20:30].toarray()

In [ ]:
axis_adata_mese_forR = axis_adata_mese.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in axis_adata_mese_forR.var_names if not name.startswith('MT')]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in axis_adata_mese_forR.var_names if not name.startswith('RP')]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in axis_adata_mese_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_haemo_genes]


# remove heat shock protein genes
non_hps_genes = [name for name in axis_adata_mese_forR.var_names if not name.startswith('HSP')]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_hps_genes]

sc.pp.filter_genes(axis_adata_mese_forR, min_cells = 10)
sc.pp.filter_genes(axis_adata_mese_forR, min_counts = 1)

In [ ]:
axis_adata_mese_forR.shape

In [ ]:
for col in axis_adata_mese_forR.obs.columns:
    if col not in [ 'MaleReproductiveAxis', 'sample']:
        del axis_adata_mese_forR.obs[col]

In [ ]:
del axis_adata_mese_forR.obsm

In [ ]:
%%R -i axis_adata_mese_forR
axis_adata_mese_forR

In [ ]:
%%R 
saveRDS(axis_adata_mese_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_mese_cytassist_downsampled.rds')

In [ ]:
wolffian_TFs_mese = ['HOXC4', 'HOXC5', 'HOXC6', 'HOXA7', 'HOXA9', 'HOXA10', 'HOXA11', 
                     'ARX', 'ALX1',  'ZBTB7C', 'FOXC1', 'RORB','MEIS2',
                     ]

In [ ]:
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_mese,var_names=wolffian_TFs_mese,groupby='binned_MaleReproductiveAxis_6_bins',
                 num_categories = 6,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_wolffian_mese_TFs_spatiallyvariable.pdf'
                )

In [ ]:
wolffian_lig_rec_mese = [ 'GRM7',  'RSPO2', 'WNT4', 'WNT5A', 'SULT1E1', 'ADRA2A', 'DKK2', 'GDF7', 'WIF1','CDH7', 
                          'TNC','RORB', 
                     ]

In [ ]:
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_mese,var_names=wolffian_lig_rec_mese,groupby='binned_MaleReproductiveAxis_6_bins',
                 num_categories = 6,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_wolffian_lig_rec_mese_spatiallyvariable.pdf'
                )

In [ ]:
axis_adata_epi = axis_adata[[i for i in axis_adata.obs_names.to_list() if i in epididymal_epi_spots], :]

In [ ]:
axis_adata_epi

In [ ]:
axis_adata_epi.obs['MaleReproductiveAxis'].hist(bins = 10)

In [ ]:
axis = bin_axis(axis_adata_epi.obs, n_bins = 6, axis_name = 'MaleReproductiveAxis')
axis['binned_MaleReproductiveAxis_6_bins'] = axis['binned_MaleReproductiveAxis_6_bins'].astype(float)
axis['binned_MaleReproductiveAxis_6_bins'] = np.round(axis['binned_MaleReproductiveAxis_6_bins'], 2)
axis_adata_epi.obs['binned_MaleReproductiveAxis_6_bins'] = axis_adata_epi.obs_names.map(axis['binned_MaleReproductiveAxis_6_bins'].to_dict())

In [ ]:
axis_adata_epi.obs['binned_MaleReproductiveAxis_6_bins'].value_counts()

In [ ]:
axis_adata_epi.write('/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_epi_cytassist.h5ad')

In [ ]:
axis_adata_epi = downsample(axis_adata_mese, 'binned_MaleReproductiveAxis_6_bins', 20)

In [ ]:
axis_adata_epi.write('/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_epi_downsampled_cytassist.h5ad')

In [ ]:
axis_adata_epi.obs['MaleReproductiveAxis'].hist(bins = 6)

In [ ]:
axis_adata_epi.X[20:30, 20:30].toarray()

In [ ]:
axis_adata_epi_forR = axis_adata_epi.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in axis_adata_epi_forR.var_names if not name.startswith('MT')]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in axis_adata_epi_forR.var_names if not name.startswith('RP')]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in axis_adata_epi_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_haemo_genes]


# remove heat shock protein genes
non_hps_genes = [name for name in axis_adata_epi_forR.var_names if not name.startswith('HSP')]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_hps_genes]

sc.pp.filter_genes(axis_adata_epi_forR, min_cells = 10)
sc.pp.filter_genes(axis_adata_epi_forR, min_counts = 1)

In [ ]:
axis_adata_epi_forR.shape

In [ ]:
for col in axis_adata_epi_forR.obs.columns:
    if col not in [ 'MaleReproductiveAxis', 'sample']:
        del axis_adata_epi_forR.obs[col]

In [ ]:
del axis_adata_epi_forR.obsm

In [ ]:
%%R -i axis_adata_epi_forR
axis_adata_epi_forR

In [ ]:
%%R 
saveRDS(axis_adata_epi_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/malereproductiveaxis_epi_cytassist_downsampled.rds')

In [ ]:
wolffian_epi_TFs = ['AR', 'ESR1', 'HNF1A', 'PITX2', 'MLXIPL']
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_epi,var_names=wolffian_epi_TFs,groupby='binned_MaleReproductiveAxis_6_bins',
                 num_categories = 6,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_wolffian_epi_TFs_spatiallyvariable.pdf'
                )

In [ ]:
wolffian_epi_partners = ['SLC1A1', 'GLS', 'LGR4', 'LGR5', 'LRP5', 'LRP6', 'FZD3', 
                        'FZD2', 'PPARA', 'PPARG', 'PPARD', 'PNMT', 'KREMEN2', 'BMPR1B', 'BMPR2', 
                        'WNT6', 'WNT9B', 'CDH7', 'ITGA1', 'ITGB6', 'ALDH1A1']
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_epi,var_names=wolffian_epi_partners,groupby='binned_MaleReproductiveAxis_6_bins',
                 num_categories = 6,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_wolffian_epi_partners_spatiallyvariable.pdf'
                )

## Plot Wolffian longitudinal axis in 10x Visium samples

In [ ]:
HCA_F_RepTsp14707486_to_plot = select_slide(adata_cytassist, 'HCA_F_RepTsp14707486', batch_key = 'sample')
HCA_F_RepTsp14707486_to_plot.obs['MaleReproductiveAxis'] = HCA_F_RepTsp14707486_to_plot.obs_names.map(axis_adata.obs['MaleReproductiveAxis'])
HCA_F_RepTsp14707486_to_plot.obs['MaleReproductiveAxis'].head()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap


# Define the colors and their corresponding positions
colors = ["#aa92c4", "#e9b8d1", "#ef8466", "#f19c6f"]

# Create the colormap
cmap_wolffian = LinearSegmentedColormap.from_list('custom_cmap', colors, N=256)

In [ ]:
sc.pl.spatial(HCA_F_RepTsp14707486_to_plot, color = 'MaleReproductiveAxis', cmap = cmap_wolffian, 
             save = 'HCA_F_RepTsp14707486_wolffian_axis')

In [ ]:
HCA_F_RepTsp14733322_to_plot = select_slide(adata_cytassist, 'HCA_F_RepTsp14733322', batch_key = 'sample')
HCA_F_RepTsp14733322_to_plot.obs['MaleReproductiveAxis'] = HCA_F_RepTsp14733322_to_plot.obs_names.map(axis_adata.obs['MaleReproductiveAxis'])
HCA_F_RepTsp14733322_to_plot.obs['MaleReproductiveAxis'].head()

In [ ]:
sc.pl.spatial(HCA_F_RepTsp14733322_to_plot, color = 'MaleReproductiveAxis', cmap = cmap_wolffian, 
              bw = True,
              alpha_img = 0.7, spot_size = 350,
             save = 'HCA_F_RepTsp14733322_wolffian_axis')